In [1]:
import sys
#sys.path.insert(0,'/home/lshtanko/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/marusy/Programming/another-fem-framework/')
sys.path.insert(0,'/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [2]:
#import h5py

from operator_assembler.assembly_interface import AssemblyInterface2D
from operator_assembler.n_to_1_ddof_allocator import Nto1Allocator2D
from grid.grid_manager import GridManager
from grid.allocator_interface import InterfaceDofAllocator2D
from grid.grid_domain import GridDomain
from operator_assembler.matrix_assembler import MatrixAssembler2D
from scipy.ndimage import imread
from grid import cell, grid_layer, grid_manager
import h5py
import numpy as np

from common import visual
from common.tests import *
from common.helpers import *
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import linalg

%matplotlib inline

#### Bad cell

In [ ]:
data_start = (16, 20)
data_shape = (4, 4)
data_end = tuple([i+j for i,j in zip(data_start, data_shape)])

h5_data = h5py.File('../../../GM_L3D/600/state/viz_GM_1.h5')['index'][0]
data = h5_data[data_start[0]:data_end[0], data_start[1]:data_end[1]]
#data = imread('/Users/marusy/Programming/model/bhi2_labelled0000.tif')[data_start[0]:data_end[0], data_start[1]:data_end[1]]

grid_domain = GridDomain(integer_size=data_shape, domain_size=(1.,1.))
gm = GridManager()
ifma = InterfaceDofAllocator2D(grid_manager=gm.fit(data=data))

gm.draw_grid()

In [3]:
cell1 = cell.Cell2D(size=(1,1), ll_vertex=(0,0))
cell2 = cell.Cell2D(size=(1,1), ll_vertex=(0,1))

cell3 = cell.Cell2D(size=(2,2), ll_vertex=(1,0))

In [4]:
layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2]], layer_number=0, index=[1,1], div_index=[1,1])
layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3]], layer_number=1, index=[1], div_index=[1])

In [5]:
gm = grid_manager.GridManager(dim=2, method='coarse2to1')
gm.grid_layers = [layer1, layer2]
ifma = InterfaceDofAllocator2D(grid_manager=gm)

In [6]:
ma = Nto1Allocator2D(grid_interface=ifma)
ma._make_ddof_index()
grid_domain = GridDomain(domain_size=(3.,2.), integer_size=(3,2))
grid_domain.make_pointwise_index(ma)

In [7]:
ifma2 = AssemblyInterface2D(allocator=ma)
MA = MatrixAssembler2D(assembly_interface=ifma2, grid_domain=grid_domain)
deep = MA.assemble()

[(0, 4), (1, 3), (2, 1), (3, 5), (4, 8), (5, 0), (6, 6), (7, 7), (8, 2)] [(0, 1), (1, 9), (2, 10), (3, 0), (4, 14), (5, 11), (6, 2), (7, 13), (8, 12)]
[(0, 4), (1, 3), (2, 1), (3, 5), (4, 8), (5, 0), (6, 6), (7, 7), (8, 2)] [(0, 1), (1, 9), (2, 10), (3, 0), (4, 14), (5, 11), (6, 2), (7, 13), (8, 12)]
{0, 1, 3, 4, 5, 8, 9, 10, 11, 14} {15, 17, 18, 19, 20, 23} [[21, 22, 16]] [[6, 7, 2, 13, 12]]
 2

In [ ]:
sine_test = grid_domain.vectorize_function(lambda x,y: np.sin(x+y))
product = grid_domain.devectorize_vector(MA.assembled * sine_test)
init = grid_domain.devectorize_function(lambda x,y: np.sin(x+y))

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_vector(MA.assembled * sine_test),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_function(lambda x,y: np.sin(x+y)),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

In [9]:
for i in grid_domain.normal_index.keys():
    print(i, end=' ')
    partial_diff_test(matrix=MA.half_glob, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

4 X derivative: 0.16666666655555584
Y derivative: 0.16127566994444165
XX derivative: -0.02777777772222334
YY derivative: -0.02718845836110879
XY_derivative: 0.005390996722222451

3 X derivative: 0.6666666672222228
Y derivative: 0.006032346388885499
XX derivative: -0.11111111138888706
YY derivative: -0.10458619618055655
XY_derivative: -0.006032346388888608

1 X derivative: 0.3333333329999999
Y derivative: 3.0531133177191805e-15
XX derivative: -0.05555555538888876
YY derivative: -0.06503437688888365
XY_derivative: 4.163336342344337e-17

5 X derivative: 2.7777777300563855e-10
Y derivative: 0.6672685811111143
XX derivative: -0.11111111118055361
YY derivative: -0.1087683224444422
XY_derivative: -0.00030095722222231447

8 X derivative: -8.888928759720471e-10
Y derivative: 0.015076902222212168
XX derivative: -0.4444444442222254
YY derivative: -0.4596157668888938
XY_derivative: -0.007538451111110081

0 X derivative: -1.0000024940060026e-09
Y derivative: 1.0443035325380379e-14
XX derivative: -0

In [ ]:
MA.half_glob[2].toarray().flatten()

In [ ]:
deep[0][-1].toarray()[2]

In [ ]:
deep[0][1][2].toarray()

In [ ]:
MA.assembled[2].toarray().flatten()[-9:]

In [ ]:
pipka[2].toarray().flatten()

In [ ]:
partial_diff_test(matrix=deep[0][-1], point=grid_domain.normal_index[2], pointnum=2, pointdict=grid_domain.pointwise_index)

In [ ]:
dist = distributed_eye_easy(ma.get_cell_list_of_ddofs(cell=cell3), axis2shape=MA.assembly_interface.get_ddof_count())

In [ ]:
pipka = dist.T * csr_matrix(MA._get_local_gg_matrix(distribution='lobatto', order=2)) * dist

In [ ]:
mm = np.array(MA.assembled_mass.sum(axis=1)).flatten()

In [ ]:
mm_sp = csr_matrix(np.diag(1/mm))

In [ ]:
op =  mm_sp * MA.assembled 

In [ ]:
grid_domain.normal_index

In [ ]:
MA.assembly_interface.allocator.get_cell_list_of_ddofs(cell2)

In [ ]:
plt.imshow(deep[0][-1].toarray(), interpolation='none')

In [ ]:
MA.I_s2b